In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -y psi4 python=3.7 -c psi4

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

# this command is needed to avoid "Loader" error.
!pip install distributed

--2022-04-24 22:45:25--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-24 22:45:25--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M   145MB/s    in 0.5s    

2022-04-24 22:45:25 (145 MB/s) - ‘Miniconda3-latest-Linux

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [3]:
psi4.set_output_file("log.txt")

psi4.set_memory('10GB')
psi4.set_num_threads(2)

mol = psi4.geometry("""
symmetry c1
0 1
O       -1.0239104333      1.3152089264     -0.0149071027                 
H       -0.0345840622      1.2807129464      0.0105245830                 
H       -1.3132760473      0.5529382075      0.5470664770                 
""")

#scf_e, wfn = psi4.optimize("B3LYP-D3BJ/6-311G(d,p)",  return_wfn = True)
#scf_e, wfn = psi4.optimize("M06-D3/6-311G(d,p)", return_wfn = True) # metal
scf_e, wfn = psi4.optimize("M06-2X-D3/6-311G(d,p)", return_wfn = True) # non-metal

Optimizer: Optimization complete!


In [4]:
!cat log.txt


  Memory set to   9.313 GiB by Python driver.
  Threads set to 2 by Python driver.

Scratch directory: /tmp/

Scratch directory: /tmp/
gradient() will perform analytic gradient computation.

*** tstart() called on a62887babb7e
*** at Sun Apr 24 22:49:54 2022

   => Loading Basis Set <=

    Name: 6-311G(D,P)
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   151 file /usr/local/share/psi4/basis/6-311g_d_p_.gbs 
    atoms 2-3 entry H          line    32 file /usr/local/share/psi4/basis/6-311g_d_p_.gbs 

   => M06-2X-D3: Empirical Dispersion <=

    Grimme's -D3 (zero-damping) Dispersion Correction
    Grimme S.; Antony J.; Ehrlich S.; Krieg H. (2010), J. Chem. Phys., 132: 154104
    Parametrisation from:    L. Goerigk, S. Grimme, Phys. Chem. Chem. Phys. 13, 6670-6688, 2011

        s6 =       1.000000
        s8 =       0.000000
       sr6 =       1.619000
    alpha6 =      14.000000
       sr8 =       1.000000


         ----------------------------------------

In [5]:
xyz = mol.save_string_xyz()
print (xyz)

0 1
 O    0.000000000002   -0.000000000000    0.065991238612
 H    0.756279797731   -0.000000000000   -0.523664422664
 H   -0.756279797770    0.000000000000   -0.523664422605



In [6]:
psi4.core.set_output_file("frq.out")

psi4.set_memory('10GB')
psi4.set_num_threads(2)

# save molden file for Avogadro (GUI)
psi4.set_options({"NORMAL_MODES_WRITE": True})

# calculation (vibration)
#scf_e, wfn = psi4.frequency("B3LYP-D3BJ/6-311G(d,p)", ref_gradient = wfn.gradient(), return_wfn = True)
#scf_e, wfn = psi4.frequency("M06-D3/6-311G(d,p)", ref_gradient = wfn.gradient(), return_wfn = True) # metal
scf_e, wfn = psi4.frequency("M06-2X-D3/6-311G(d,p)", ref_gradient = wfn.gradient(), return_wfn = True) # non-metal

 7 displacements needed.
 1 2 3 4 5 6 7


In [7]:
!cat frq.out


  Memory set to   9.313 GiB by Python driver.
  Threads set to 2 by Python driver.
hessian() using ref_gradient to assess stationary point.

  Based on options and gradient (rms=1.55E-05), recommend projecting translations and projecting rotations.
hessian() will perform frequency computation by finite difference of analytic gradients.

         ----------------------------------------------------------
                                   FINDIF
                     R. A. King and Jonathon Misiewicz
         ---------------------------------------------------------

  Using finite-differences of gradients to determine vibrational frequencies and 
  normal modes. Resulting frequencies are only valid at stationary points.
    Generating geometries for use with 3-point formula.
    Displacement size will be 5.00e-03.
    Number of atoms is 3.
    Number of irreps is 1.
    Number of SALCs is 3.
    Translations projected? 1. Rotations projected? 1.
    Index of SALCs per irrep:
     1 :  

In [32]:
ld = open('frq.out')
lines = ld.readlines()
for line in lines:
    if line.find("Total") >= 0 and line.find("H,") >= 0:
        H = line.split()[6]
    if line.find("Total") >= 0 and line.find("G,") >= 0:
        G = line.split()[7]
print("Total H [Ha], Enthalpy at  298.15 [K] =",H)
print("Total G [Ha], Free enthalpy at  298.15 [K] =",G)
#!awk '{if($1="Total" && $2=="H,"){print $7}}' frq.out
#!awk '{if($1=="Total" && $2=="G,"){print $8}}' frq.out

Total H [Ha], Enthalpy at  298.15 [K] = -76.38693535
Total G [Ha], Free enthalpy at  298.15 [K] = -76.40835204


In [33]:
wfn.frequencies().to_array()
wfn.hessian().to_array()

array([[ 6.98103793e-01,  0.00000000e+00, -5.45322384e-09,
        -3.49051901e-01,  0.00000000e+00,  2.72148521e-01,
        -3.49051892e-01,  0.00000000e+00, -2.72148516e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-5.45322384e-09,  0.00000000e+00,  4.73244560e-01,
         2.13407816e-01,  0.00000000e+00, -2.36622282e-01,
        -2.13407811e-01,  0.00000000e+00, -2.36622278e-01],
       [-3.49051901e-01,  0.00000000e+00,  2.13407816e-01,
         3.79906649e-01,  0.00000000e+00, -2.42778169e-01,
        -3.08547480e-02,  0.00000000e+00,  2.93703531e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.72148521e-01,  0.00000000e+00, -2.36622282e-01,
        -2.42778169e-01,  0.00000000e+00,  2.244054

In [34]:
import numpy as np
print(f'Frequencies:\n{np.array(wfn.frequencies())}')
print(f'Frequencies:\n{np.array(wfn.hessian())}')

Frequencies:
[1626.7262636  3887.11061876 3983.5564284 ]
Frequencies:
[[ 6.98103793e-01  0.00000000e+00 -5.45322384e-09 -3.49051901e-01
   0.00000000e+00  2.72148521e-01 -3.49051892e-01  0.00000000e+00
  -2.72148516e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-5.45322384e-09  0.00000000e+00  4.73244560e-01  2.13407816e-01
   0.00000000e+00 -2.36622282e-01 -2.13407811e-01  0.00000000e+00
  -2.36622278e-01]
 [-3.49051901e-01  0.00000000e+00  2.13407816e-01  3.79906649e-01
   0.00000000e+00 -2.42778169e-01 -3.08547480e-02  0.00000000e+00
   2.93703531e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.72148521e-01  0.00000000e+00 -2.36622282e-01 -2.42778169e-01
   0.00000000e+00  2.24405418e-01 -2.93703524e-02  0.00000000e+00
   1.22168643e-02]
 [-3.49051892e-01  0.000